In [156]:
import numpy as np
import pandas as pd
import random

In [157]:
with open("pcv.txt", "r") as arquivo:
    distancias = []
    cidades = arquivo.readlines()
    for i in range(len(cidades)-1):
        distancias.append(cidades[i + 1].strip('\n'" ").split(" ")[1:])
    #for i in range(len(cidades)):
     #   cidades[i] = cidades[i].strip('\n'" ")

print(distancias)

[['0', '1', '2', '1', '1', '2'], ['1', '0', '7', '1', '4', '3'], ['2', '7', '0', '3', '1', '1'], ['1', '1', '3', '0', '8', '1'], ['1', '4', '1', '8', '0', '1'], ['2', '3', '1', '1', '1', '0']]


In [158]:
class Cidades:
    def _init_(self, cidades = []):
        self.cidades = cidades    
    
    def ler_entrada(self):
        with open("pcv.txt", "r") as arquivo:
            arquivo = arquivo.readlines()
            for i in range(len(arquivo)):
                arquivo[i] = arquivo[i].strip('\n'" ")
        return arquivo
        
    def obter_cidades(self):
        cidades = []
        arquivo = self.ler_entrada()
        cidades.append(arquivo[0].split(" "))
        return cidades[0]
    
    def obter_distancias(self):
        distancias = []
        arquivo = self.ler_entrada()
        for i in range(len(arquivo)-1):
            distancias.append(arquivo[i + 1].split(" ")[1:])
        return distancias


In [185]:
c = Cidades()

In [186]:
c.ler_entrada()


['A B C D E F',
 'A 0 1 2 1 1 2',
 'B 1 0 7 1 4 3',
 'C 2 7 0 3 1 1',
 'D 1 1 3 0 8 1',
 'E 1 4 1 8 0 1',
 'F 2 3 1 1 1 0']

In [272]:
cidades= c.obter_cidades()

In [284]:

class Distancia():
    def _init_(self, cidades):
        self.cidades = cidades

    def obter_distancia_entre_duas_cidades(self, cidade_inicial, cidade_final):
       c =  Cidades()
       index_cidade_inicial = self.cidades.index(cidade_inicial)
       index_cidade_final = self.cidades.index(cidade_final)
       distancias = c.obter_distancias()
       distancia =  int(distancias[index_cidade_inicial][index_cidade_final])
       return distancia

In [285]:
class Individuo():
    def __init__(self, cidades, geracao = 0):
        self.cidades = cidades
        self.cromossomo = []
        self.geracao = geracao
        self.probabilidade = 0
        
        listaCidades = []
        listaCidades.extend(self.cidades)
        print (self.cidades)
                
        while len(listaCidades) > 0 :
            cidade_index =  random.randrange(len(listaCidades))           
            self.cromossomo.append(listaCidades[cidade_index])
            listaCidades.pop(cidade_index)
            
        print (self.cromossomo)  
    
    def fitness(self):
        soma_distancia = 0
        
        for i in range(len(self.cromossomo)-1):
            d = Distancia()
            d.cidades = self.cidades
            cidade_inicial = self.cromossomo[i]
            cidade_final = self.cromossomo[i+1]
            soma_distancia += d.obter_distancia_entre_duas_cidades(cidade_inicial, cidade_final)
          
        print(soma_distancia)  
        return soma_distancia

In [286]:
c.obter_distancias()

[['0', '1', '2', '1', '1', '2'],
 ['1', '0', '7', '1', '4', '3'],
 ['2', '7', '0', '3', '1', '1'],
 ['1', '1', '3', '0', '8', '1'],
 ['1', '4', '1', '8', '0', '1'],
 ['2', '3', '1', '1', '1', '0']]

In [287]:
i=Individuo(cidades)

['A', 'B', 'C', 'D', 'E', 'F']
['D', 'C', 'E', 'A', 'B', 'F']


In [288]:
i.fitness()

9


9

In [263]:
a = [['0', '1', '2', '1', '1', '2'], ['1', '0', '7', '1', '4', '3'], ['2', '7', '0', '3', '1', '1'], ['1', '1', '3', '0', '8', '1'], ['1', '4', '1', '8', '0', '1'], ['2', '3', '1', '1', '1', '0']]

In [154]:
class AlgoritimoGenetico():
    def __init__(self, cidades, tamanho_populacao = 50,  geracao = 50):
        self.tamanho_populacao  = tamanho_populacao
        self.geracao = geracao
        self.cidades = cidades
        self.populacao = []
        self.probabilidade = 0
        self.tamanho_roleta = 0
       

    #gera população inicial
    def init_populacao(self, cidades):
        for i in range(self.tamanho_populacao):
            self.populacao.append(Individuo(cidades))
                        
    def ordena_populacao(self):
        self.populacao =  sorted(self.populacao, key=lambda populacao:populacao.distancia_total, reverse = False)
    
    def pressao_seletiva(self, n):
        N = self.tamanho_populacao
        self.probabilidade  =  (N-n)/(N-1)
        
    def selecao_roda_roleta_com_ordenacao(self):        
        self.probabilidade = [(self.populacao[i], self.pressao_seletiva(i+1) for i in self.tamanho_população)]
        self.tamanho_roleta  =  sum([i[1] for i in self.probabilidade])
        
   
    def seleciona_pais(self):
        selecao =[]
        self.selecao_roda_roleta_com_ordenacao()
        for i in self.tamanho_populacao:
            ponto_aleatorio= random.uniform(0, self.tamanho_roleta)
            acc = 0
            for self.populacao[i].cromossomo, self.populacao[i].fitness in self.probabilidade:
                ponto_aleatorio= random.uniform(0, self.tamanho_roleta)
                if ponto_aleatorio <= acc + self.populacao[i].fitness:
                    selecao +=self.populacao[i].cromossomo
                    break
                acc +=self.populacao[i].fitness
                
        return selecao
    
    def cruzamento(self, pais):
        filhos = []
        for i in range(len(self.populacao)/2):    
            pai1 = pais[i]
            pai2 = pais[i+1]
            gene_index =  random.randrange(len(pai1))
            filho1= pai1[-1:gene_index] + pai2[gene_index :]
            filho2= pai2[-1:gene_index] + pai1[gene_index :]
            filhos+=filho1
            filhos+=filho2
            
        return filhos            
        

    def init_solucao(self, cidades):
        self.init_populacao(cidades)
        
        for individuo in self.populacao:
            individuo.fitness()
        
        #ordena população por fitness
        self.ordena_populacao()
        
        #print resultado
        melhorIndividuo = self.populacao[0]
        print("G: %s -> Distancia total: %s Cromossomo: %s" % (
            melhorIndividuo.geracao,
            melhorIndividuo.distancia_total,
            melhorIndividuo.cromossomo
        ))

        for i in range(self.geracao):     
            pais = self.seleciona_pais    
            filhos =  self.cruzamento(pais)  
            #mutação
            #seleciona sobreviventes 
            #calcula fitness
            
          
  

SyntaxError: invalid syntax (1575477958.py, line 24)